## Female Athlete Nutritional Requirements

It is surprisingly difficult to find reliable information about the recommended nutrient intake for female athletes, possibly because most athlete nutritional studies are conducted on men.

Basic research about endurance athletes:
* In general, female endurance athletes need one gram of protein per pound of body weight per day for effective muscle repair, growth, and recovery.
* During the strength and power phases of training, female athletes should aim for 1.0 to 1.2 grams of protein/pound.
* For endurance phases, female athletes need 0.8 to 1.0 grams of protein per pound
* For non-training or easy days, female athletes should take in about 0.75 to 0.8 grams of protein per pound

Based on this information, we made the following changes to nutritional requirements between the typical USDA guidance and guidance for a female endurance athlete:
* 80 grams of protein (instead of 46 g) based on the [Academy of Nutrition and Dietetics, Dietitians of Canada, and the American College of Sports Medicine](https://www.sciencedirect.com/science/article/pii/S221226721501802X?via%3Dihub)
* 22 mg of iron (instead of 18 mg), based on [US Military Guidance](https://nap.nationalacademies.org/catalog/11610/mineral-requirements-for-military-personnel-levels-needed-for-cognitive-and)
* 2745 g calories (instead of 2000), 488 g of carbohydrates (instead of 130 g), and 25 mcg of Vitamin D (instead of 15), based on [this NIH study, Recommendations for Healthy Nutrition in Female Endurance Runners](https://pmc.ncbi.nlm.nih.gov/articles/PMC4443719/)

In [2]:
# Initial set-ups

%pip install eep153_tools
%pip install python_gnupg
%pip install -U gspread_pandas
#load in file from class, but our own version where we have added nutrition information for athletes
def format_id(id,zeropadding=0):
    """Nice string format for any id, string or numeric.

    Optional zeropadding parameter takes an integer
    formats as {id:0z} where
    """
    if pd.isnull(id) or id in ['','.']: return None

    try:  # If numeric, return as string int
        return ('%d' % id).zfill(zeropadding)
    except TypeError:  # Not numeric
        return id.split('.')[0].strip().zfill(zeropadding)
    except ValueError:
        return None

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [6]:
import pandas as pd
import re 
from eep153_tools.sheets import read_sheets
data_url_nutrients = "https://docs.google.com/spreadsheets/d/1GTo423_gUJe1Von9jypWAbC0zSQ7WGegAWPuRi7eJAI/edit?gid=415594035#gid=415594035"

In [7]:
rda = read_sheets(data_url, sheet="rda")

rda = rda.set_index("Nutrient")

rda.columns, rda.head()

(Index(['Nutrient Type', 'Unit', 'Constraint Type', 'Female_19_30',
        'Female_endurance', 'Male_19_30', 'Male_endurance'],
       dtype='object'),
               Nutrient Type  Unit Constraint Type  Female_19_30  \
 Nutrient                                                          
 Energy                Macro  kcal             RDA        2000.0   
 Protein               Macro     g             RDA          46.0   
 Carbohydrate          Macro     g             RDA         130.0   
 Dietary Fiber         Macro     g             RDA          28.0   
 Linoleic Acid         Macro     g              AI          12.0   
 
                Female_endurance  Male_19_30  Male_endurance  
 Nutrient                                                     
 Energy                   2800.0      2400.0          2700.0  
 Protein                    80.0        56.0             NaN  
 Carbohydrate              488.0       130.0           350.0  
 Dietary Fiber              28.0        33.6          

## Copying Jack's code and trying to make a dietary recommendation!

In [8]:
#create recipes df
recipes = read_sheets(data_url, sheet="recipes")
recipes = (recipes
           .assign(parent_foodcode = lambda df: df["parent_foodcode"].apply(format_id),
                   ingred_code = lambda df: df["ingred_code"].apply(format_id))
           .rename(columns={"parent_desc": "recipe"}))
recipes.head()
#recipes.shape

,parent_foodcode,recipe,ingred_code,ingred_desc,ingred_wt
0,11340000,"Imitation milk, non-soy, sweetened",43543,"Milk, imitation, non-soy",100.0
1,11460150,"Yogurt, frozen, NS as to flavor, lowfat milk",1298,"Yogurt, frozen, flavors other than chocolate, ...",100.0
2,11460160,"Yogurt, frozen, chocolate, lowfat milk",1117,"Yogurt, plain, low fat, 12 grams protein per 8...",81.8
3,11460160,"Yogurt, frozen, chocolate, lowfat milk",19166,"Cocoa, dry powder, unsweetened, processed with...",5.2
4,11460160,"Yogurt, frozen, chocolate, lowfat milk",19335,"Sugars, granulated",13.0


In [ ]:
#check the data types
recipes.dtypes

In [ ]:
#List of non-vegan keywords
NON_VEGAN_KEYWORDS = [
    "beef", "pork", "chicken", "turkey", "fish", "seafood", "shellfish", "shrimp", "crab","crabs",
    "lamb", "goat", "duck", "goose", "tuna", "salmon", "cod", "bacon", "ham",
    "shellfish", "lobster", "mussels", "oysters", "scallops", "octopus", "eel",
    "organ meat", "milk","Eggnog" "cheese", "butter", "cream","ice cream", "yogurt", "whey",
    "casein", "lactose", "ghee", "buttermilk", "egg", "eggs", "mayo", "mayonnaise", "albumen",
    "albumin", "lysozyme", "ovomucoid", "ovomucin", "ovovitellin", "honey",
    "bee pollen", "royal jelly", "propolis", "shellac", "confectioner’s glaze",
    "carmine", "cochineal", "lard", "tallow", "suet", "gelatin", "collagen",
    "isinglass", "bone broth", "bone stock", "fish sauce", "oyster sauce",
    "shrimp paste", "worcestershire sauce", "anchovies", "rennet", "pepsin",
    "bone char", "vitamin d3", "lanolin", "omega-3 fish oil", "caseinate",
    "lecithin (egg)", "cysteine", "l-cysteine", "glycerin (animal)",
    "glycerol (animal)", "stearic acid (animal)", "tallowate", "sodium tallowate",
    "capric acid", "caprylic acid"
]

#this partal match: "milkshake" or "eggroll" will get flagged (since "milk" or "egg" is in the keyword list).
NON_VEGAN_PATTERN = re.compile(
    '|'.join(map(re.escape, NON_VEGAN_KEYWORDS)),
    re.IGNORECASE
)


def filter_vegan_ingredients(df: pd.DataFrame) -> pd.DataFrame:
    
    df[["recipe", "ingred_desc"]] = df[["recipe", "ingred_desc"]].astype(str)

    
    mask = ~(df["recipe"].str.contains(NON_VEGAN_PATTERN, na=False, regex=True) |
             df["ingred_desc"].str.contains(NON_VEGAN_PATTERN, na=False, regex=True))
    
    return df[mask]

In [ ]:
vegan_recipes = filter_vegan_ingredients(recipes)
vegan_recipes